In [2]:
#Paquetes para Preprocesamiento
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk import ngrams
from nltk import regexp_tokenize          
import unicodedata
import demoji
from nltk.stem import SnowballStemmer
import os


emo_unicode = os.path.join('..','0_Data','emo_unicode.py')

%run $emo_unicode

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def clean_text_sentiment_analysis(texto):
    texto = remover_url(texto)
    texto = remover_usuario(texto)
    texto = demoji.replace(texto) #transformamos los emojis en palabras
    texto = convert_emoticons(texto)
    texto = remover_tag_html(texto)
    texto = remover_espacio_extra(texto)
    texto = remover_punto(texto)
    texto = stemmizar_expandir_abreviaturas_contracciones(texto)
    return texto

In [3]:
def limpiar_texto(texto):
    texto = minuscula(texto)
    texto = remover_url(texto)
    texto = remover_usuario(texto)
    texto = remover_emoji(texto)
    texto = remover_caracter_especial(texto)
    texto = remover_tag_html(texto)
    texto = remover_acento(texto)
    texto = remover_punto_numero(texto)
    texto = remover_espacio_extra(texto)
    texto = lematizar_remover_stop_words_abreviaturas_contracciones(texto)
    return texto

In [4]:
def clean_text_simple(texto):
    texto = minuscula(texto)
    texto = remover_url(texto)
    texto = remover_usuario(texto)
    texto = remover_punto(texto)
    return texto

In [5]:
# Transformar el texto a minuscula

def minuscula(texto):

    return texto.lower()

#Remover URL

def remover_url(texto):

    url = re.compile(r'https?://\S+|www\.\S+')

    return url.sub(r'', texto)

def convert_emoticons(texto):
    for emot in EMOTICONS:
        texto = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), texto)
    return texto

#Remover hashtags que aparecen (#hashtag)
def remover_hashtags(texto):
    return re.sub("#[A-Za-z0-9]+","", texto)


#Remover Usuarios que aparecen (@usuario)

def remover_usuario(text):

    text = re.sub(r"\@[A-Za-z0-9]+", "", text)

    return text

# Remover Emoji

def remover_emoji(texto):

    emoji_patrones = re.compile(

        '['

        u'\U0001F600-\U0001F64F' 

        u'\U0001F300-\U0001F5FF' 

        u'\U0001F680-\U0001F6FF' 

        u'\U0001F1E0-\U0001F1FF' 

        u'\U00002702-\U000027B0'

        u'\U000024C2-\U0001F251'

        ']+',

        flags=re.UNICODE)

    return emoji_patrones.sub(r'', texto)

#Expandir las Abreviaturas

abreviaturas = {

    "$" : " dollar ",

    "€" : " euro ",

    "4ao" : "for adults only",

    "a.m" : "before midday",

    "a3" : "anytime anywhere anyplace",

    "aamof" : "as a matter of fact",

    "acct" : "account",

    "adih" : "another day in hell",

    "afaic" : "as far as i am concerned",

    "afaict" : "as far as i can tell",

    "afaik" : "as far as i know",

    "afair" : "as far as i remember",

    "afk" : "away from keyboard",

    "app" : "application",

    "approx" : "approximately",

    "apps" : "applications",

    "asap" : "as soon as possible",

    "asl" : "age, sex, location",

    "atk" : "at the keyboard",

    "ave." : "avenue",

    "aymm" : "are you my mother",

    "ayor" : "at your own risk", 

    "b&b" : "bed and breakfast",

    "b+b" : "bed and breakfast",

    "b.c" : "before christ",

    "b2b" : "business to business",

    "b2c" : "business to customer",

    "b4" : "before",

    "b4n" : "bye for now",

    "b@u" : "back at you",

    "bae" : "before anyone else",

    "bak" : "back at keyboard",

    "bbbg" : "bye bye be good",

    "bbc" : "british broadcasting corporation",

    "bbias" : "be back in a second",

    "bbl" : "be back later",

    "bbs" : "be back soon",

    "be4" : "before",

    "bfn" : "bye for now",

    "blvd" : "boulevard",

    "bout" : "about",

    "brb" : "be right back",

    "bros" : "brothers",

    "brt" : "be right there",

    "bsaaw" : "big smile and a wink",

    "btw" : "by the way",

    "bwl" : "bursting with laughter",

    "c/o" : "care of",

    "cet" : "central european time",

    "cf" : "compare",

    "cia" : "central intelligence agency",

    "csl" : "can not stop laughing",

    "cu" : "see you",

    "cul8r" : "see you later",

    "cv" : "curriculum vitae",

    "cwot" : "complete waste of time",

    "cya" : "see you",

    "cyt" : "see you tomorrow",

    "dae" : "does anyone else",

    "dbmib" : "do not bother me i am busy",

    "diy" : "do it yourself",

    "dm" : "direct message",

    "dwh" : "during work hours",

    "e123" : "easy as one two three",

    "eet" : "eastern european time",

    "eg" : "example",

    "embm" : "early morning business meeting",

    "encl" : "enclosed",

    "encl." : "enclosed",

    "etc" : "and so on",

    "faq" : "frequently asked questions",

    "fawc" : "for anyone who cares",

    "fb" : "facebook",

    "fc" : "fingers crossed",

    "fig" : "figure",

    "fimh" : "forever in my heart", 

    "ft." : "feet",

    "ft" : "featuring",

    "ftl" : "for the loss",

    "ftw" : "for the win",

    "fwiw" : "for what it is worth",

    "fyi" : "for your information",

    "g9" : "genius",

    "gahoy" : "get a hold of yourself",

    "gal" : "get a life",

    "gcse" : "general certificate of secondary education",

    "gfn" : "gone for now",

    "gg" : "good game",

    "gl" : "good luck",

    "glhf" : "good luck have fun",

    "gmt" : "greenwich mean time",

    "gmta" : "great minds think alike",

    "gn" : "good night",

    "g.o.a.t" : "greatest of all time",

    "goat" : "greatest of all time",

    "goi" : "get over it",

    "gps" : "global positioning system",

    "gr8" : "great",

    "gratz" : "congratulations",

    "gyal" : "girl",

    "h&c" : "hot and cold",

    "hp" : "horsepower",

    "hr" : "hour",

    "hrh" : "his royal highness",

    "ht" : "height",

    "ibrb" : "i will be right back",

    "ic" : "i see",

    "icq" : "i seek you",

    "icymi" : "in case you missed it",

    "idc" : "i do not care",

    "idgadf" : "i do not give a damn fuck",

    "idgaf" : "i do not give a fuck",

    "idk" : "i do not know",

    "ie" : "that is",

    "i.e" : "that is",

    "ifyp" : "i feel your pain",

    "IG" : "instagram",

    "iirc" : "if i remember correctly",

    "ilu" : "i love you",

    "ily" : "i love you",

    "imho" : "in my humble opinion",

    "imo" : "in my opinion",

    "imu" : "i miss you",

    "iow" : "in other words",

    "irl" : "in real life",

    "j4f" : "just for fun",

    "jic" : "just in case",

    "jk" : "just kidding",

    "jsyk" : "just so you know",

    "l8r" : "later",

    "lb" : "pound",

    "lbs" : "pounds",

    "ldr" : "long distance relationship",

    "lmao" : "laugh my ass off",

    "lmfao" : "laugh my fucking ass off",

    "lol" : "laughing out loud",

    "ltd" : "limited",

    "ltns" : "long time no see",

    "m8" : "mate",

    "mf" : "motherfucker",

    "mfs" : "motherfuckers",

    "mfw" : "my face when",

    "mofo" : "motherfucker",

    "mph" : "miles per hour",

    "mr" : "mister",

    "mrw" : "my reaction when",

    "ms" : "miss",

    "mte" : "my thoughts exactly",

    "nagi" : "not a good idea",

    "nbc" : "national broadcasting company",

    "nbd" : "not big deal",

    "nfs" : "not for sale",

    "ngl" : "not going to lie",

    "nhs" : "national health service",

    "nrn" : "no reply necessary",

    "nsfl" : "not safe for life",

    "nsfw" : "not safe for work",

    "nth" : "nice to have",

    "nvr" : "never",

    "nyc" : "new york city",

    "oc" : "original content",

    "og" : "original",

    "ohp" : "overhead projector",

    "oic" : "oh i see",

    "omdb" : "over my dead body",

    "omg" : "oh my god",

    "omw" : "on my way",

    "p.a" : "per annum",

    "p.m" : "after midday",

    "pm" : "prime minister",

    "poc" : "people of color",

    "pov" : "point of view",

    "pp" : "pages",

    "ppl" : "people",

    "prw" : "parents are watching",

    "ps" : "postscript",

    "pt" : "point",

    "ptb" : "please text back",

    "pto" : "please turn over",

    "qpsa" : "what happens", 

    "ratchet" : "rude",

    "rbtl" : "read between the lines",

    "rlrt" : "real life retweet", 

    "rofl" : "rolling on the floor laughing",

    "roflol" : "rolling on the floor laughing out loud",

    "rotflmao" : "rolling on the floor laughing my ass off",

    "rt" : "retweet",

    "ruok" : "are you ok",

    "sfw" : "safe for work",

    "sk8" : "skate",

    "smh" : "shake my head",

    "sq" : "square",

    "srsly" : "seriously", 

    "ssdd" : "same stuff different day",

    "tbh" : "to be honest",

    "tbs" : "tablespooful",

    "tbsp" : "tablespooful",

    "tfw" : "that feeling when",

    "thks" : "thank you",

    "tho" : "though",

    "thx" : "thank you",

    "tia" : "thanks in advance",

    "til" : "today i learned",

    "tl;dr" : "too long i did not read",

    "tldr" : "too long i did not read",

    "tmb" : "tweet me back",

    "tntl" : "trying not to laugh",

    "ttyl" : "talk to you later",

    "u" : "you",

    "u2" : "you too",

    "u4e" : "yours for ever",

    "utc" : "coordinated universal time",

    "w/" : "with",

    "w/o" : "without",

    "w8" : "wait",

    "wassup" : "what is up",

    "wb" : "welcome back",

    "wtf" : "what the fuck",

    "wtg" : "way to go",

    "wtpa" : "where the party at",

    "wuf" : "where are you from",

    "wuzup" : "what is up",

    "wywh" : "wish you were here",

    "yd" : "yard",

    "ygtr" : "you got that right",

    "ynk" : "you never know",

    "zzz" : "sleeping bored and tired"

}

def expandir_abreviatura(texto,mapping = abreviaturas):

    texto = ' '.join([mapping[t] if t in mapping else t for t in texto.split(" ")])

    return texto

#Expandir las Contracciones

contracciones_mapeo = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", 

                       "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 

                       "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", 

                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 

                       "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",

                       "mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 

                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 

                       "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",

                       "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 

                       "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 

                       "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 

                       "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 

                       "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 

                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", 

                       "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", 

                       "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", 

                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }


def expandir_contraccion(texto,mapping = contracciones_mapeo):

    specials =["’", "‘", "´", "`"]

    for s in specials:

        texto = texto.replace(s,"'")

    

    texto = ' '.join([mapping[t] if t in mapping else t for t in texto.split(" ")])

    return texto

#Remover Caracteres Especiales

def remover_caracter_especial(texto):

    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 

    return re.sub(pat, '', texto)

# Remover los tags HTML

def remover_tag_html(texto):

    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

    return re.sub(html, '', texto)

#Remover acentos

def remover_acento(texto):

    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    return texto

#Remover Puntos

def remover_punto(texto):
    
    return texto.translate(str.maketrans(string.punctuation,' '*(len(string.punctuation))))

    #texto = ''.join([c for c in texto if c not in string.punctuation])

    #return texto

#Remover Numeros

def remover_numero(texto):

    #texto = ''.join([i for i in texto if not i.isdigit()])

    #return texto
    
    return texto.translate(str.maketrans('','',string.digits))

def remover_punto_numero(texto):

    import string

    texto = ''.join([c for c in texto if c not in string.punctuation and not c.isdigit()])

    return texto

#Remover espacios en Blanco (extras/tabs)

def remover_espacio_extra(texto):

    pattern = r'^\s*|\s\s*'

    return re.sub(pattern, ' ', texto).strip()

#Remover Stop-Word

def remover_stop_word(texto):

    return " ".join ([word for word in word_tokenize(texto) if not word in stopwords.words('english')])

#Lematizar

def lematizar(texto):
    lemma = WordNetLemmatizer()
    return " ".join([lemma.lemmatize(word) for word in word_tokenize(texto)])


In [6]:
#Usamos esta version del tokenizador por la ventaja en performance que tiene
lemma = WordNetLemmatizer()
#cacheamos los stopwords porque la diferencia en performance es ENORME
cachedStopWords = stopwords.words("english")
specials =["’", "‘", "´", "`"]
stemmer = SnowballStemmer("english")

def stemmizar_expandir_abreviaturas_contracciones(texto, mapping = contracciones_mapeo, mapping_abreviaturas = abreviaturas):

    tokenizacion = regexp_tokenize(texto, '\s', gaps=True)

    for s in specials:

        texto = texto.replace(s,"'")
    
    return " ".join([stemmer.stem(mapping_abreviaturas[word] if word in mapping_abreviaturas else mapping[word] if word in mapping else word) for word in tokenizacion])


#Lematizar y remover Stop-Words
def lematizar_remover_stop_words_abreviaturas_contracciones(texto, mapping = contracciones_mapeo, mapping_abreviaturas = abreviaturas):

    tokenizacion = regexp_tokenize(texto, '\s', gaps=True)

    for s in specials:

        texto = texto.replace(s,"'")
    
    return " ".join([lemma.lemmatize(mapping_abreviaturas[word] if word in mapping_abreviaturas else mapping[word] if word in mapping else word) for word in tokenizacion if word not in cachedStopWords])

In [7]:
#Usamos esta version del tokenizador por la ventaja en performance que tiene
lemma = WordNetLemmatizer()
#cacheamos los stopwords porque la diferencia en performance es ENORME
cachedStopWords = stopwords.words("english")
specials =["’", "‘", "´", "`"]
stemmer = SnowballStemmer("english")

def stemmizar_expandir_abreviaturas_contracciones(texto, mapping = contracciones_mapeo, mapping_abreviaturas = abreviaturas):

    tokenizacion = regexp_tokenize(texto, '\s', gaps=True)

    for s in specials:

        texto = texto.replace(s,"'")
    
    return " ".join([stemmer.stem(mapping_abreviaturas[word] if word in mapping_abreviaturas else mapping[word] if word in mapping else word) for word in tokenizacion])


#Lematizar y remover Stop-Words
def lematizar_remover_stop_words_abreviaturas_contracciones(texto, mapping = contracciones_mapeo, mapping_abreviaturas = abreviaturas):

    tokenizacion = regexp_tokenize(texto, '\s', gaps=True)

    for s in specials:

        texto = texto.replace(s,"'")
    
    return " ".join([lemma.lemmatize(mapping_abreviaturas[word] if word in mapping_abreviaturas else mapping[word] if word in mapping else word) for word in tokenizacion if word not in cachedStopWords])

In [8]:
#Usamos esta version del tokenizador por la ventaja en performance que tiene
lemma = WordNetLemmatizer()
#cacheamos los stopwords porque la diferencia en performance es ENORME
cachedStopWords = stopwords.words("english")
specials =["’", "‘", "´", "`"]
stemmer = SnowballStemmer("english")

def stemmizar_expandir_abreviaturas_contracciones(texto, mapping = contracciones_mapeo, mapping_abreviaturas = abreviaturas):

    tokenizacion = regexp_tokenize(texto, '\s', gaps=True)

    for s in specials:

        texto = texto.replace(s,"'")
    
    return " ".join([stemmer.stem(mapping_abreviaturas[word] if word in mapping_abreviaturas else mapping[word] if word in mapping else word) for word in tokenizacion])


#Lematizar y remover Stop-Words
def lematizar_remover_stop_words_abreviaturas_contracciones(texto, mapping = contracciones_mapeo, mapping_abreviaturas = abreviaturas):

    tokenizacion = regexp_tokenize(texto, '\s', gaps=True)

    for s in specials:

        texto = texto.replace(s,"'")
    
    return " ".join([lemma.lemmatize(mapping_abreviaturas[word] if word in mapping_abreviaturas else mapping[word] if word in mapping else word) for word in tokenizacion if word not in cachedStopWords])